## 판다스를 활용한 실전 데이터 분석

캐글의 타이타닉([Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic/)) 데이터를 더 깊게 분석합니다.

데이터 사이언티스트로서 데이터를 분석할 때 가장 필수적인 스킬은 판다스([Pandas](https://pandas.pydata.org/))입니다. 데이터 분석가는 언제나 주변 사람들의 요청(BI팀, CS팀 등)에 맞게 데이터를 뽑아내 그 통계치를 제공해줘야 하는데, 판다스 스킬이 부족하면 주어진 시간 내에 이를 제공해 줄 수 없습니다.

또한 스스로 데이터를 분석할 때도, 하고 싶은 실험이나 추출하고 싶은 데이터가 있는데 이를 판다스로 처리 못 하는 상황이 종종 발생합니다. (일명 '머리에 비해 손이 못 따라온다.'고 표현합니다) 이런 문제가 누적되면 다른 동료들에 비해 분석 속도나 실험 속도가 크게 뒤처질 수 밖에 없습니다.

이런 문제가 생기지 않기 위해, **모든 데이터 사이언티스트는 데이터를 능숙하게 다룰 수 있는 판다스 스킬을 필수적으로 보유하고 있어야 합니다.**

이번 노트북에는 타이타닉 데이터를 활용하여, 현업에서 충분히 발생할 만한 사례를 모아 총 19개의 문제를 만들어 보았습니다. 주어진 시간 내에 모든 문제를 해결해보세요. 테스트 결과, 현업에서 데이터 사이언티스트로 일 하고 있는 분들은 아무리 늦어도 반나절(3~4시간) 내에는 모든 문제를 풀 수 있었습니다. 즉, 3시간 안에 모든 문제를 풀 수 있다면 합격입니다.

문제를 풀 때 다른 자료를 참고하거나, 구글에 검색하는 것 모두 허용합니다. (문제 중에는 구글에 검색하지 않으면 풀 수 없는 문제도 몇 개 준비해놓았습니다) 관련 자료는 [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html) 를 강력 추천합니다.

In [1]:
import pandas as pd

### 데이터 읽어오기

In [2]:
# train.csv 파일을 읽어옵니다. 여기서 PassengerId라는 컬럼을 인덱스(index)로 지정한 뒤, train 변수에 할당합니다.
# 변수에 할당한 결과값을 판다스 전문 용어로 데이터프레임(DataFrame)이라고 부릅니다.
train = pd.read_csv("data/titanic/train.csv", index_col="PassengerId")

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# head()로 train 데이터의 상위 5개를 출력합니다.
train.head()

(891, 11)


Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin Embarked  
PassengerId                                                          
1                1      0         A/5 21171   7.2500   NaN        S  
2                1      0          PC 17599  71.2833   C85        C  
3                0      0  STON/O2. 3101282   7.9250   NaN        S  
4                1      0            113803  53.1000  C123        S  
5                0      0            373450   8.0500   NaN        S

### 데이터 정리 + 기본 분석

**1. 타이타닉의 train 데이터에서 1) 전체 생존률과 2) 생존자의 총 인원수, 사망자의 총 인원수를 출력해주세요.**

1번(생존률)의 경우 약 38.3%가 나와야 하며, 2번(인원수)의 경우 생존자의 총 인원수는 342명, 사망자의 총 인원수는 549명이 나와야 합니다.

In [3]:
# 타이타닉의 train데이터에서 Survived 컬럼의 평균을 구합니다.
# 그 결과를 survived_rate라는 이름의 변수에 저장합니다.
survived_rate = train["Survived"].mean()

# survived_rate는 현재 0.0 ~ 1.0 사이의 값을 갖습니다.
# 하지만 퍼센티지(%)는 0 ~ 100.0 사이의 값을 가지므로, survived_rate에 100을 곱해줍니다.
survived_rate = survived_rate * 100

# survived_rate를 출력합니다. 결과는 38.4%가 나와야 합니다.
print(f"생존률 = {survived_rate:.1f}%")

생존률 = 38.4%


In [4]:
# pandas의 value_counts를 활용하여 생존자의 총 인원수와 사망자의 총 인원수를 출력합니다.
# 생존자의 총 인원수(1)은 342명, 사망자의 총 인원수(0)는 549명이 나와야 합니다.
train["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

** 2. Survived 컬럼에 들어가 있는 값을 쉬운 표현으로 바꿔주세요. **

Survived 컬럼에는 0(사망)이라는 값과 1(생존)이라는 값이 있습니다. 이 표현은 직관적이지 않기 때문에, 데이터 분석을 원활하게 하기 위해서는 사람이 읽기 쉬운 표현을 쓰는 것이 좋습니다.

In [5]:
# Survived 컬럼의 상위 5개의 값을 출력합니다.
# 결과값은 0과 1이 나오는데, Survived 컬럼에 대한 사전 설명(가령 0이 어떤 값을 나타내는지, 1이 어떤 값을 나타내는지)
# 을 듣지 않으면 이 값이 어떠한 의미를 가지는지 직관적으로 이해하기 어렵습니다.
train["Survived"].head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

사례로 **Survived(humanized)**라는 새로운 컬럼을 만드는 방법이 있습니다. 이 컬럼에는 0(사망), 1(생존)이 아닌, Perish(사망), Survived(생존)이라는 값이 들어가 있다면 좋겠습니다. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>
        <th>Survived</th>
        <th>Survived(humanized)</th>
      </tr>

      <tr>
        <th>PassengerId</th>
        <th></th>
        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>
        <td>0</td>
        <td>Perish</td>
      </tr>

      <tr>
        <th>2</th>
        <td>1</td>
        <td>Survived</td>
      </tr>

      <tr>
        <th>3</th>
        <td>1</td>
        <td>Survived</td>
      </tr>

      <tr>
        <th>4</th>
        <td>1</td>
        <td>Survived</td>
      </tr>

      <tr>
        <th>5</th>
        <td>0</td>
        <td>Perish</td>
      </tr>
    </tbody>
  </table>

In [6]:
# 먼저 Survived 컬럼이 0인 승객을 색인합니다. 이후 Survived(humanized)라는 이름의
# 새로운 컬럼을 만들어 여기에 Perish 라는 값을 넣습니다.
train.loc[train["Survived"] == 0, "Survived(humanized)"] = "Perish"

# 비슷하게 Survived 컬럼이 1인 승객을 찾아 Survived(humanized)에 Survived라는 값을 넣습니다.
train.loc[train["Survived"] == 1, "Survived(humanized)"] = "Survived"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# Survived 컬럼과 Survived(humanized) 컬럼 두 개를 출력하여 비교합니다.
train[["Survived", "Survived(humanized)"]].head()

(891, 12)


Survived Survived(humanized)
PassengerId                              
1                   0              Perish
2                   1            Survived
3                   1            Survived
4                   1            Survived
5                   0              Perish

또한 이번에는 Survived 컬럼이 아닌 아닌 새롭게 만든 Survived(humanized) 컬럼으로 생존자의 총 인원수와 사망자의 총 인원수를 출력해 주세요. 앞서 사용한 ```value_counts```를 그대로 사용하면 될 것 같습니다.

In [7]:
# pandas의 value_counts를 활용하여 생존자의 총 인원수와 사망자의 총 인원수를 출력합니다.
# 여기서 생존 여부는 Survived가 아닌 Survived(humanized) 컬럼을 사용합니다.
# 생존자의 총 인원수(Survived)은 342명, 사망자의 총 인원수(Perish)는 549명이 나와야 합니다.
train["Survived(humanized)"].value_counts()

Perish      549
Survived    342
Name: Survived(humanized), dtype: int64

** 3. Pclass 컬럼에 들어가 있는 값을 읽기 쉬운 표현으로 바꿔주세요. **

Pclass도 마찬가지로 1, 2, 3이라는 표현은 직관적이지 않기 때문에, 사람이 이해하기 쉬운 표현으로 바꿔주고 싶습니다.

In [8]:
# pandas의 pivot_table을 활용하여 Pclass별 생존률을 출력합니다.
# 여기서 Pclass값이 1, 2, 3이 나오는데, Pclass 컬럼에 대한 사전 설명을 듣지 않으면 이해하기 어렵습니다.
# 그러므로 Pclass값을 조금 더 직관적으로 바꿔준다면 pivot_table로 분석하기 편할 것입니다.
pd.pivot_table(data=train, index="Pclass", values="Survived")

Survived
Pclass          
1       0.629630
2       0.472826
3       0.242363

이번에는 **Pclass(humanized)**라는 새로운 컬럼을 만들어주세요. 이 컬럼에는 1, 2, 3이 아닌 First Class, Business, Economy 라는 값이 들어가 있다면 좋겠습니다. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Pclass</th>

        <th>Pclass(humanized)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>3</td>

        <td>Economy</td>
      </tr>

      <tr>
        <th>2</th>

        <td>1</td>

        <td>First Class</td>
      </tr>

      <tr>
        <th>3</th>

        <td>3</td>

        <td>Economy</td>
      </tr>

      <tr>
        <th>4</th>

        <td>1</td>

        <td>First Class</td>
      </tr>

      <tr>
        <th>5</th>

        <td>3</td>

        <td>Economy</td>
      </tr>
    </tbody>
  </table>

In [9]:
# 먼저 Pclass 컬럼이 1인 승객을 색인합니다. 이후 Pclass(humanized)라는 이름의
# 새로운 컬럼을 만들어 여기에 "First Class" 라는 값을 넣습니다.
train.loc[train["Pclass"] == 1, "Pclass(humanized)"] = "First Class"

# 비슷하게 Pclass 컬럼이 2인 승객을 찾아 Pclass(humanized)에 "Business" 라는 값을 넣습니다.
train.loc[train["Pclass"] == 2, "Pclass(humanized)"] = "Business"

# 비슷하게 Pclass 컬럼이 3인 승객을 찾아 Pclass(humanized)에 "Economy" 라는 값을 넣습니다.
train.loc[train["Pclass"] == 3, "Pclass(humanized)"] = "Economy"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# Pclass 컬럼과 Pclass(humanized) 컬럼 두 개를 출력하여 비교합니다.
train[["Pclass", "Pclass(humanized)"]].head()

(891, 13)


Pclass Pclass(humanized)
PassengerId                          
1                 3           Economy
2                 1       First Class
3                 3           Economy
4                 1       First Class
5                 3           Economy

또한 이번에는 Pclass가 아닌 새롭게 만든 Pclass(humanized) 컬럼으로 pivot_table을 만들어주세요. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>Pclass(humanized)</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>Business</th>

        <td>0.472826</td>
      </tr>

      <tr>
        <th>Economy</th>

        <td>0.242363</td>
      </tr>

      <tr>
        <th>First Class</th>

        <td>0.629630</td>
      </tr>
    </tbody>
  </table>

In [10]:
# pandas의 pivot_table을 활용하여 Pclass별 생존률을 출력합니다.
# 하지만 이번에는 Pclass 컬럼이 아닌 Pclass(humanized) 컬럼을 사용합니다.
# 이전에 비해서 훨씬 더 직관적으로 생존률을 확인할 수 있습니다.
pd.pivot_table(data=train, index="Pclass(humanized)", values="Survived")

Survived
Pclass(humanized)          
Business           0.472826
Economy            0.242363
First Class        0.629630

** 4. Embarked 컬럼에 들어가 있는 값을 읽기 쉬운 표현으로 바꿔주세요. **

Embarked 컬럼도 마찬가지로 C, S, Q라는 표현은 직관적이지 않습니다. 저라면 사람이 조금 더 이해하기 쉽게끔 C는 Cherbourg 라는 표현으로, S는 Southampton 이라는 표현으로, 그리고 Q는 Queenstown 이라는 표현으로 바꾸겠습니다.

In [11]:
# pandas의 pivot_table을 활용하여 Embarked 별 생존률을 출력합니다.
# 여기서도 Embarked 컬럼이 C, S, Q라는 다소 직관적이지 않은 값이 나옵니다.
# 그러므로 Embarked 컬럼의 값도 Pclass 처럼 직관적으로 바꿔주고 싶습니다.
pd.pivot_table(data=train, index="Embarked", values="Survived")

Survived
Embarked          
C         0.553571
Q         0.389610
S         0.336957

Survived(humanized), Pclass(humanized)와 마찬가지로, Embarked 컬럼도 **Embarked(humanized)**라는 이름의 새로운 컬럼을 만들어주세요. 이 컬럼에는 C, S, Q가 아닌 Cherbourg, Southampton, Queenstown이라는 값이 들어갑니다. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Embarked</th>

        <th>Embarked(humanized)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>S</td>

        <td>Southampton</td>
      </tr>

      <tr>
        <th>2</th>

        <td>C</td>

        <td>Cherbourg</td>
      </tr>

      <tr>
        <th>3</th>

        <td>S</td>

        <td>Southampton</td>
      </tr>

      <tr>
        <th>4</th>

        <td>S</td>

        <td>Southampton</td>
      </tr>

      <tr>
        <th>5</th>

        <td>S</td>

        <td>Southampton</td>
      </tr>
    </tbody>
  </table>

In [12]:
# 먼저 Embarked 컬럼이 C인 승객을 색인합니다. 이후 Embarked(humanized)라는 이름의
# 새로운 컬럼을 만들어 여기에 "Cherbourg" 라는 값을 넣습니다.
train.loc[train["Embarked"] == "C", "Embarked(humanized)"] = "Cherbourg"

# 비슷하게 Embarked 컬럼이 2인 승객을 찾아 Embarked(humanized)에 "Southampton" 이라는 값을 넣습니다.
train.loc[train["Embarked"] == "S", "Embarked(humanized)"] = "Southampton"

# 비슷하게 Embarked 컬럼이 3인 승객을 찾아 Embarked(humanized)에 "Queenstown" 이라는 값을 넣습니다.
train.loc[train["Embarked"] == "Q", "Embarked(humanized)"] = "Queenstown"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# Embarked 컬럼과 Embarked(humanized) 컬럼 두 개를 출력하여 비교합니다.
train[["Embarked", "Embarked(humanized)"]].head()

(891, 14)


Embarked Embarked(humanized)
PassengerId                             
1                  S         Southampton
2                  C           Cherbourg
3                  S         Southampton
4                  S         Southampton
5                  S         Southampton

그리고 마찬가지로 이번에도 Embarked가 아닌 새롭게 만든 Embarked(humanized) 컬럼으로 pivot_table을 만들어주세요. 여기에는 Cherbourg, Southampton, 그리고 Queenstown 에 탑승한 승객의 생존률이 출력됩니다. 최종적으로는 다음의 결과가 나와야 합니다.

 <table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>Embarked(humanized)</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>Cherbourg</th>

        <td>0.553571</td>
      </tr>

      <tr>
        <th>Queenstown</th>

        <td>0.389610</td>
      </tr>

      <tr>
        <th>Southampton</th>

        <td>0.336957</td>
      </tr>
    </tbody>
  </table>

In [13]:
# pandas의 pivot_table을 활용하여 Embarked별 생존률을 출력합니다.
# 마찬가지로 Embarked 컬럼이 아닌 Embarked(humanized) 컬럼을 사용합니다.
# 이전에 비해서 훨씬 더 직관적으로 생존률을 확인할 수 있습니다.
pd.pivot_table(data=train, index="Embarked(humanized)", values="Survived")

Survived
Embarked(humanized)          
Cherbourg            0.553571
Queenstown           0.389610
Southampton          0.336957

** 5. Embarked 컬럼과 Pclass 컬럼을 One Hot Encoding 해주세요.**

앞서 작성한 코드를 활용하여 Embarked 컬럼과 Pclass 컬럼을 충분히 분석한 것 같습니다. 이번에는 이 두 개의 컬럼을 의사결정나무(Decision Tree)와 같은 머신러닝(Machine Learning) 알고리즘에 넣을 수 있도록 [One Hot Encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)을 해줄려고 합니다. 먼저 Embarked 컬럼을 바탕으로, 다음의 세 개의 컬럼을 새롭게 만들고 싶습니다.

  1. **Embarked(Cherbourg)** - Embarked 컬럼 안에 있는 값이 C면 True, 아니면 False가 들어갑니다.
  1. **Embarked(Southampton)** - Embarked 컬럼 안에 있는 값이 S면 True, 아니면 False가 들어갑니다.
  1. **Embarked(Queenstown)** - Embarked 컬럼 안에 있는 값이 Q면 True, 아니면 False가 들어갑니다.

최종적으로는 다음의 결과가 나와야 합니다.

 <table border="1" class="dataframe" style="float: left">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Embarked</th>

        <th>Embarked(humanized)</th>

        <th>Embarked(Cherbourg)</th>

        <th>Embarked(Southampton)</th>

        <th>Embarked(Queenstown)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>2</th>

        <td>C</td>

        <td>Cherbourg</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>3</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>4</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>5</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>6</th>

        <td>Q</td>

        <td>Queenstown</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>7</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>8</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>9</th>

        <td>S</td>

        <td>Southampton</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>10</th>

        <td>C</td>

        <td>Cherbourg</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>
    </tbody>
  </table>

Pclass 컬럼도 비슷하게 다음의 세 개의 컬럼이 만들어져야 합니다.

  1. **Pclass(First Class)** - Pclass 컬럼 안에 있는 값이 1이면 True, 아니면 False가 들어갑니다.
  1. **Pclass(Business)** - Pclass 컬럼 안에 있는 값이 2면 True, 아니면 False가 들어갑니다.
  1. **Pclass(Economy)** - Pclass 컬럼 안에 있는 값이 3이면 True, 아니면 False가 들어갑니다.
  
최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Pclass</th>

        <th>Pclass(humanized)</th>

        <th>Pclass(First Class)</th>

        <th>Pclass(Business)</th>

        <th>Pclass(Economy)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>3</td>

        <td>Economy</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>2</th>

        <td>1</td>

        <td>First Class</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>3</th>

        <td>3</td>

        <td>Economy</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>4</th>

        <td>1</td>

        <td>First Class</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>5</th>

        <td>3</td>

        <td>Economy</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>6</th>

        <td>3</td>

        <td>Economy</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>7</th>

        <td>1</td>

        <td>First Class</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>8</th>

        <td>3</td>

        <td>Economy</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>9</th>

        <td>3</td>

        <td>Economy</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>10</th>

        <td>2</td>

        <td>Business</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>
    </tbody>
  </table>

Embarked 컬럼과 Pclass 컬럼을 활용하여 앞서 설명한 여섯 개의 컬럼을 만들어주세요.

In [14]:
# 먼저 Embarked 컬럼의 값이 C인 승객을 색인합니다. 결과적으로 Embarked가 C이면 True, 아니면 False인 리스트가 나옵니다.
# 이 결과를 Embarked(Cherbourg)라는 이름의 새로운 컬럼에 대입합니다.
train["Embarked(Cherbourg)"] = train["Embarked"] == "C"

# 비슷한 방식으로 Embarked(Southampton)라는 이름의 새로운 컬럼을 추가합니다.
# 여기에는 Embarked 컬럼의 값이 S이면 True, 아니면 False가 들어갑니다.
train["Embarked(Southampton)"] = train["Embarked"] == "S"

# 비슷한 방식으로 Embarked(Queenstown)이라는 이름의 새로운 컬럼을 추가합니다.
# 여기에는 Embarked 컬럼의 값이 Q이면 True, 아니면 False가 들어갑니다.
train["Embarked(Queenstown)"] = train["Embarked"] == "Q"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# Embarked 컬럼과 기타 연관 컬럼들을 출력하여 비교합니다.
train[["Embarked", "Embarked(humanized)", "Embarked(Cherbourg)", "Embarked(Southampton)", "Embarked(Queenstown)"]].head(10)

(891, 17)


Embarked Embarked(humanized)  Embarked(Cherbourg)  \
PassengerId                                                     
1                  S         Southampton                False   
2                  C           Cherbourg                 True   
3                  S         Southampton                False   
4                  S         Southampton                False   
5                  S         Southampton                False   
6                  Q          Queenstown                False   
7                  S         Southampton                False   
8                  S         Southampton                False   
9                  S         Southampton                False   
10                 C           Cherbourg                 True   

             Embarked(Southampton)  Embarked(Queenstown)  
PassengerId                                               
1                             True                 False  
2                            False                 False  
3                             True                 False  
4                             True                 False  
5                             True                 False  
6                            False                  True  
7                             True                 False  
8                             True                 False  
9                             True                 False  
10                           False                 False

In [15]:
# Pclass의 One Hot Encoding도 Pclass와 비슷합니다.
# 먼저 Pclass 컬럼의 값이 1인 승객을 색인합니다. 결과적으로 Pclass가 1이면 True, 아니면 False인 리스트가 나옵니다.
# 이 결과를 Pclass(First Class)라는 이름의 새로운 컬럼에 대입합니다.
train["Pclass(First Class)"] = train["Pclass"] == 1

# 비슷한 방식으로 Pclass(Business)라는 이름의 새로운 컬럼을 추가합니다.
# 여기에는 Pclass 컬럼의 값이 2이면 True, 아니면 False가 들어갑니다.
train["Pclass(Business)"] = train["Pclass"] == 2

# 비슷한 방식으로 Pclass(Economy)라는 이름의 새로운 컬럼을 추가합니다.
# 여기에는 Pclass 컬럼의 값이 3이면 True, 아니면 False가 들어갑니다.
train["Pclass(Economy)"] = train["Pclass"] == 3

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# Pclass 컬럼과 기타 연관 컬럼들을 출력하여 비교합니다.
train[["Pclass", "Pclass(humanized)", "Pclass(First Class)", "Pclass(Business)", "Pclass(Economy)"]].head(10)

(891, 20)


Pclass Pclass(humanized)  Pclass(First Class)  Pclass(Business)  \
PassengerId                                                                    
1                 3           Economy                False             False   
2                 1       First Class                 True             False   
3                 3           Economy                False             False   
4                 1       First Class                 True             False   
5                 3           Economy                False             False   
6                 3           Economy                False             False   
7                 1       First Class                 True             False   
8                 3           Economy                False             False   
9                 3           Economy                False             False   
10                2          Business                False              True   

             Pclass(Economy)  
PassengerId                   
1                       True  
2                      False  
3                       True  
4                      False  
5                       True  
6                       True  
7                      False  
8                       True  
9                       True  
10                     False

### 나이(Age) 컬럼 분석

**6. 나이(Age) 컬럼에서 다음의 정보를 출력해주세요.**

  * 평균(mean)
  * 분산(standard deviation, 이하 std) 
  * 가장 나이가 많은 사람.
  * 가장 나이가 적은 사람.
  
가령 평균은 약 29.7세, 분산(std)은 약 14.52가 나와야 합니다.

In [16]:
# 나이(Age) 컬럼에서 mean 함수를 통해 평균 나이를 구합니다.
# 평균 나이가 약 29.7세라는 것을 알 수 있습니다.
train["Age"].mean()

29.69911764705882

In [17]:
# 나이(Age) 컬럼에서 std 함수를 통해 나이의 표준 편차를 구합니다.
# 나이의 분산이 약 14.52세라는 것을 알 수 있습니다.
train["Age"].std()

14.526497332334044

In [18]:
# 나이(Age) 컬럼에서 min 함수를 통해 나이의 최소치를 구합니다.
# 타이타닉호에 탑승한 가장 어린 승객은 약 0.42세(생후 4개월 정도)라는 것을 알 수 있습니다.
train["Age"].min()

0.41999999999999998

In [19]:
# 나이(Age) 컬럼에서 max 함수를 통해 나이의 최대치 구합니다.
# 타이타닉호에 탑승한 가장 나이가 많은 승객은 80세라는 것을 알 수 있습니다.
train["Age"].max()

80.0

또는 판다스의 [describe](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html)를 활용하면 한 줄의 코드로 평균, 분산, 최소치, 최대치를 볼 수 있습니다.

In [20]:
# 나이(Age) 컬럼에 대해 describe 함수를 사용합니다.
# 이 함수는 특정 컬럼의 평균, 분산, 최대치, 최소치와 같은 기초적인 통계치를 보여줍니다.
train["Age"].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

**7. 객실 등급별 나이(Age) 컬럼의 평균을 보여주세요.**

이번에는 전체 평균이 아닌 객실 등급(Pclass)별 평균을 보고 싶습니다.

가령 전체 승객의 평균 나이는 약 29.7세이지만, 1등급 승객의 평균 나이는 약 38.2세가 나와야 합니다. 비슷한 방식으로 2등급과 3등급 승객의 평균 나이를 알 수 있다면 좋겠습니다.

In [21]:
# Pclass가 1등급인 승객만 색인해서 가져온 뒤, 이를 Pclass1이라는 변수에 할당합니다.
pclass1 = train[train["Pclass"] == 1]

# 1등급 승객의 평균 나이를 구합니다.
pclass1["Age"].mean()

38.233440860215055

In [22]:
# Pclass가 2등급인 승객만 색인해서 가져온 뒤, 이를 Pclass2이라는 변수에 할당합니다.
pclass2 = train[train["Pclass"] == 2]

# 2등급 승객의 평균 나이를 구합니다.
pclass2["Age"].mean()

29.87763005780347

In [23]:
# Pclass가 3등급인 승객만 색인해서 가져온 뒤, 이를 Pclass3이라는 변수에 할당합니다.
pclass3 = train[train["Pclass"] == 3]

# 3등급 승객의 평균 나이를 구합니다.
pclass3["Age"].mean()

25.14061971830986

또는 판다스의 [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)를 활용하면 한 줄의 코드로 1, 2, 3등급 승객의 평균 나이를 가져올 수 있습니다.

In [24]:
# 타이타닉 데이터를 Pclass 기준으로 그룹화합니다.
# 이렇게 하면 Pclass의 세 종류(1등급, 2등급, 3등급)마다 따로따로 연산을 할 수 있습니다.
# 이후 나이(Age) 컬럼의 평균(mean)을 구하면 1, 2, 3등급마다의 평균 나이가 나옵니다.
train.groupby("Pclass")["Age"].mean()

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

**8. 나이를 일정 구역으로 나눠서, 구역마다의 생존률을 보여주세요.**

이번에는 나이(Age)별 생존률을 확인하고 싶습니다. 다만 나이 컬럼은 숫자이기 때문에, 그대로 쓰지 않고 일정 구역으로 나눈 뒤 생존률의 통계를 내는 것이 보기 편할 것입니다. 그러므로 나이 컬럼을 다음의 세 구역으로 나눕니다.

  1. 나이가 15세 미만인 승객.
  2. 나이가 15세 이상이고 30세 미만인 승객.
  3. 나이가 30세 이상인 승객.

최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Age</th>

        <th>AgeType</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>22.0</td>

        <td>Medium</td>
      </tr>

      <tr>
        <th>2</th>

        <td>38.0</td>

        <td>Old</td>
      </tr>

      <tr>
        <th>3</th>

        <td>26.0</td>

        <td>Medium</td>
      </tr>

      <tr>
        <th>4</th>

        <td>35.0</td>

        <td>Old</td>
      </tr>

      <tr>
        <th>5</th>

        <td>35.0</td>

        <td>Old</td>
      </tr>

      <tr>
        <th>6</th>

        <td>NaN</td>

        <td>NaN</td>
      </tr>

      <tr>
        <th>7</th>

        <td>54.0</td>

        <td>Old</td>
      </tr>

      <tr>
        <th>8</th>

        <td>2.0</td>

        <td>Young</td>
      </tr>

      <tr>
        <th>9</th>

        <td>27.0</td>

        <td>Medium</td>
      </tr>

      <tr>
        <th>10</th>

        <td>14.0</td>

        <td>Young</td>
      </tr>
    </tbody>
  </table>

또한, 위 조건에서 1번, 2번, 3번 구역에 해당하는 승객의 평균 생존률을 구하고 싶습니다.

가령 1번 구역(나이가 15세 미만)에 해당하는 승객의 평균 생존률은 약 57.7%가 나와야 합니다.

In [25]:
# 나이가 15세 미만인 승객을 색인한 뒤, AgeType이라는 새로운 컬럼에 "Young"이라는 값을 넣습니다.
train.loc[train["Age"] < 15, "AgeType"] = "Young"

# 비슷하게 나이가 15세 이상 30세 미만인 승객의 AgeType에는 "Medium"이라는 값을 넣습니다.
train.loc[(train["Age"] >= 15) & (train["Age"] < 30), "AgeType"] = "Medium"

# 비슷하겍 30세 이상인 승객의 AgeType에는 "Old"이라는 값을 넣습니다.
train.loc[train["Age"] >= 30, "AgeType"] = "Old"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 나이(Age) 컬럼과 AgeType 컬럼을 출력하여 비교합니다.
train[["Age", "AgeType"]].head(10)

(891, 21)


Age AgeType
PassengerId              
1            22.0  Medium
2            38.0     Old
3            26.0  Medium
4            35.0     Old
5            35.0     Old
6             NaN     NaN
7            54.0     Old
8             2.0   Young
9            27.0  Medium
10           14.0   Young

In [26]:
# 타이타닉 데이터를 AgeType 기준으로 그룹화합니다.
# 이후 생존 여부(Survived) 컬럼의 평균(mean)을 구하면 Young, Medium, Old 마다의 평균 생존률이 나옵니다.
train.groupby("AgeType")["Survived"].mean()

AgeType
Medium    0.362745
Old       0.406061
Young     0.576923
Name: Survived, dtype: float64

**9. 나이가 비어있는 승객과 비어있지 않은 승객의 생존률 차이를 보여주세요.**

이번에는 다른 방식으로 생존률의 차이를 보겠습니다. 타이타닉 데이터의 나이(Age) 컬럼을 자세히 보면 나이가 비어있는 데이터가 있습니다. 판다스에서는 이를 NaN(Not a Number의 약자)으로 표현합니다.

타이타닉 데이터에서 나이 컬럼이 비어있는 승객과 비어있지 않은 승객의 생존률을 각각 찾아서 출력해주세요.

In [27]:
# isnull 함수를 활용해 나이 컬럼이 비어있는 승객만 색인합니다.
# 이 데이터에서 AgeBlank라는 새로운 컬럼을 만든 뒤, 여기에 "Blank"라는 값을 넣습니다.
train.loc[train["Age"].isnull(), "AgeBlank"] = "Blank"

# 비슷한 방식으로 notnull 함수를 활용하여 AgeBlank 컬럼에 "Not Blank"라는 값을 넣습니다.
train.loc[train["Age"].notnull(), "AgeBlank"] = "Not Blank"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 나이(Age) 컬럼과 AgeBlank 컬럼을 출력하여 비교합니다.
train[["Age", "AgeBlank"]].head(10)

(891, 22)


Age   AgeBlank
PassengerId                 
1            22.0  Not Blank
2            38.0  Not Blank
3            26.0  Not Blank
4            35.0  Not Blank
5            35.0  Not Blank
6             NaN      Blank
7            54.0  Not Blank
8             2.0  Not Blank
9            27.0  Not Blank
10           14.0  Not Blank

In [28]:
# 타이타닉 데이터를 AgeBlank 기준으로 그룹화합니다.
# 이후 생존 여부(Survived) 컬럼의 평균(mean)을 구하면 Blank, Not Blank 마다의 평균 생존률이 나옵니다.
train.groupby("AgeBlank")["Survived"].mean()

AgeBlank
Blank        0.293785
Not Blank    0.406162
Name: Survived, dtype: float64

**10. Pclass별 나이(Age)의 평균을 구한 뒤 빈 값에 채워주세요.**

이번에는 나이(Age) 컬럼의 빈 값을 채우고 싶습니다. 일반적으로 가장 많이 하는 방식은 나이의 평균(mean)값을 구한 뒤 이를 빈 값에 채워넣는 것입니다. 하지만 이번에는 다른 방식으로 빈 값을 채우고 싶은데, 바로 객실 등급(Pclass)에 따라 다르게 나이의 빈 값을 채워주고 싶습니다. 가령

  1. 객실 등급(Pclass)이 1등급인 승객의 평균 나이를 구해서, 해당 승객 중 나이(Age)컬럼값이 비어있는 승객을 찾아 빈 나이 값을 채워줍니다.
  2. 객실 등급(Pclass)이 2등급인 승객의 평균 나이를 구해서, 해당 승객 중 나이(Age)컬럼값이 비어있는 승객을 찾아 빈 나이 값을 채워줍니다.
  3. 객실 등급(Pclass)이 3등급인 승객의 평균 나이를 구해서, 해당 승객 중 나이(Age)컬럼값이 비어있는 승객을 찾아 빈 나이 값을 채워줍니다.
  
위와 같은 방식을 사용하면, 단순히 전체 평균을 사용하는 것 보다 조금 더 원래 값에 근접하게 평균을 채워줄 수 있을 것 같습니다. 최종적으로는 다음의 결과가 나와야 합니다.

 <table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Pclass</th>

        <th>Age</th>

        <th>Age(fill)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>3</td>

        <td>22.0</td>

        <td>22.00000</td>
      </tr>

      <tr>
        <th>2</th>

        <td>1</td>

        <td>38.0</td>

        <td>38.00000</td>
      </tr>

      <tr>
        <th>3</th>

        <td>3</td>

        <td>26.0</td>

        <td>26.00000</td>
      </tr>

      <tr>
        <th>4</th>

        <td>1</td>

        <td>35.0</td>

        <td>35.00000</td>
      </tr>

      <tr>
        <th>5</th>

        <td>3</td>

        <td>35.0</td>

        <td>35.00000</td>
      </tr>

      <tr>
        <th>6</th>

        <td>3</td>

        <td>NaN</td>

        <td>25.14062</td>
      </tr>

      <tr>
        <th>7</th>

        <td>1</td>

        <td>54.0</td>

        <td>54.00000</td>
      </tr>

      <tr>
        <th>8</th>

        <td>3</td>

        <td>2.0</td>

        <td>2.00000</td>
      </tr>

      <tr>
        <th>9</th>

        <td>3</td>

        <td>27.0</td>

        <td>27.00000</td>
      </tr>

      <tr>
        <th>10</th>

        <td>2</td>

        <td>14.0</td>

        <td>14.00000</td>
      </tr>

      <tr>
        <th>11</th>

        <td>3</td>

        <td>4.0</td>

        <td>4.00000</td>
      </tr>

      <tr>
        <th>12</th>

        <td>1</td>

        <td>58.0</td>

        <td>58.00000</td>
      </tr>

      <tr>
        <th>13</th>

        <td>3</td>

        <td>20.0</td>

        <td>20.00000</td>
      </tr>

      <tr>
        <th>14</th>

        <td>3</td>

        <td>39.0</td>

        <td>39.00000</td>
      </tr>

      <tr>
        <th>15</th>

        <td>3</td>

        <td>14.0</td>

        <td>14.00000</td>
      </tr>

      <tr>
        <th>16</th>

        <td>2</td>

        <td>55.0</td>

        <td>55.00000</td>
      </tr>

      <tr>
        <th>17</th>

        <td>3</td>

        <td>2.0</td>

        <td>2.00000</td>
      </tr>

      <tr>
        <th>18</th>

        <td>2</td>

        <td>NaN</td>

        <td>29.87763</td>
      </tr>

      <tr>
        <th>19</th>

        <td>3</td>

        <td>31.0</td>

        <td>31.00000</td>
      </tr>

      <tr>
        <th>20</th>

        <td>3</td>

        <td>NaN</td>

        <td>25.14062</td>
      </tr>
    </tbody>
  </table>

In [29]:
# 타이타닉 데이터를 Pclass 기준으로 그룹화한 뒤, 나이(Age) 컬럼의 평균을 구합니다.
# 이 결과를 mean_age_by_pclass 라는 변수에 할당합니다.
mean_age_by_pclass = train.groupby("Pclass")["Age"].mean()
mean_age_by_pclass

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

In [30]:
# Age 컬럼에 바로 값을 채워주는 것도 좋지만, 가능한 원본은 유지한 채 사본에다가 작업하는 것을 추천합니다.
# 그러므로 Age(fill) 이라는 새로운 컬럼을 만든 뒤, 이 컬럼의 빈 값을 채워줄 것입니다.
train["Age(fill)"] = train["Age"]

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 객실 등급(Pclass), 나이(Age), 그리고 Age(fill) 컬럼을 출력하여 비교합니다.
train[["Pclass","Age", "Age(fill)"]].head(30)

(891, 23)


Pclass   Age  Age(fill)
PassengerId                         
1                 3  22.0       22.0
2                 1  38.0       38.0
3                 3  26.0       26.0
4                 1  35.0       35.0
5                 3  35.0       35.0
6                 3   NaN        NaN
7                 1  54.0       54.0
8                 3   2.0        2.0
9                 3  27.0       27.0
10                2  14.0       14.0
11                3   4.0        4.0
12                1  58.0       58.0
13                3  20.0       20.0
14                3  39.0       39.0
15                3  14.0       14.0
16                2  55.0       55.0
17                3   2.0        2.0
18                2   NaN        NaN
19                3  31.0       31.0
20                3   NaN        NaN
21                2  35.0       35.0
22                2  34.0       34.0
23                3  15.0       15.0
24                1  28.0       28.0
25                3   8.0        8.0
26                3  38.0       38.0
27                3   NaN        NaN
28                1  19.0       19.0
29                3   NaN        NaN
30                3   NaN        NaN

In [31]:
# 객실 등급(Pclass)이 1등급이고 나이(Age) 컬럼값이 비어있는 승객을 색인합니다.
# 이 승객의 Age(fill)에 평균 1등급 승객의 평균 나이를 채워넣습니다.
train.loc[(train["Pclass"] == 1) & (train["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[1]

# 비슷한 원리로 객실 등급(Pclass)이 2등급인 승객도 비슷한 방식으로 빈 나이값을 채워넣습니다.
train.loc[(train["Pclass"] == 2) & (train["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[2]

# 객실 등급(Pclass)이 3등급인 승객도 비슷한 방식으로 빈 나이값을 채워넣습니다.
train.loc[(train["Pclass"] == 3) & (train["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[3]

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 객실 등급(Pclass), 나이(Age), 그리고 Age(fill) 컬럼을 출력하여 비교합니다.
train[["Pclass","Age", "Age(fill)"]].head(30)

(891, 23)


Pclass   Age  Age(fill)
PassengerId                         
1                 3  22.0   22.00000
2                 1  38.0   38.00000
3                 3  26.0   26.00000
4                 1  35.0   35.00000
5                 3  35.0   35.00000
6                 3   NaN   25.14062
7                 1  54.0   54.00000
8                 3   2.0    2.00000
9                 3  27.0   27.00000
10                2  14.0   14.00000
11                3   4.0    4.00000
12                1  58.0   58.00000
13                3  20.0   20.00000
14                3  39.0   39.00000
15                3  14.0   14.00000
16                2  55.0   55.00000
17                3   2.0    2.00000
18                2   NaN   29.87763
19                3  31.0   31.00000
20                3   NaN   25.14062
21                2  35.0   35.00000
22                2  34.0   34.00000
23                3  15.0   15.00000
24                1  28.0   28.00000
25                3   8.0    8.00000
26                3  38.0   38.00000
27                3   NaN   25.14062
28                1  19.0   19.00000
29                3   NaN   25.14062
30                3   NaN   25.14062

In [32]:
# 나이(Age) 컬럼값이 비어있는 승객만 가져온 뒤,
# 이 승객의 객실 등급(Pclass), 나이(Age), 그리고 Age(fill) 컬럼을 출력하여 비교합니다.
train.loc[train["Age"].isnull(), ["Pclass","Age", "Age(fill)"]].head(10)

Pclass  Age  Age(fill)
PassengerId                        
6                 3  NaN  25.140620
18                2  NaN  29.877630
20                3  NaN  25.140620
27                3  NaN  25.140620
29                3  NaN  25.140620
30                3  NaN  25.140620
32                1  NaN  38.233441
33                3  NaN  25.140620
37                3  NaN  25.140620
43                3  NaN  25.140620

### SibSp, Parch 컬럼 분석

**11. 타이타닉호에 동승한 형제, 자매, 배우자(SibSp)도 없고, 부모와 자식(Parch)도 없는 사람을 구해주세요.**

해당 사용자를 싱글(Single)이라고 가정하겠습니다. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>SibSp</th>

        <th>Parch</th>

        <th>Single</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>1</td>

        <td>0</td>

        <td>False</td>
      </tr>

      <tr>
        <th>2</th>

        <td>1</td>

        <td>0</td>

        <td>False</td>
      </tr>

      <tr>
        <th>3</th>

        <td>0</td>

        <td>0</td>

        <td>True</td>
      </tr>

      <tr>
        <th>4</th>

        <td>1</td>

        <td>0</td>

        <td>False</td>
      </tr>

      <tr>
        <th>5</th>

        <td>0</td>

        <td>0</td>

        <td>True</td>
      </tr>
    </tbody>
  </table>

또한 싱글(Single)인 사람과 그렇지 않은 사람간의 생존률의 차이도 알고 싶습니다. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>Single</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>False</th>

        <td>0.505650</td>
      </tr>

      <tr>
        <th>True</th>

        <td>0.303538</td>
      </tr>
    </tbody>
  </table>

In [33]:
# SibSp가 0이고 Parch가 0이면 True, 아니면 False인 리스트를 생성합니다.
# 이 리스트를 Single이라는 이름의 새로운 컬럼에 집어넣습니다.
train["Single"] = (train["SibSp"] == 0) & (train["Parch"] == 0)

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# SibSp, Parch, 그리고 Single을 출력하여 비교합니다.
train[["SibSp", "Parch", "Single"]].head()

(891, 24)


SibSp  Parch  Single
PassengerId                      
1                1      0   False
2                1      0   False
3                0      0    True
4                1      0   False
5                0      0    True

In [34]:
# pandas의 pivot_table을 활용하여 Single 여부에 따른 생존률을 출력합니다.
# Single 컬럼의 값이 True일 경우의 생존률과, False일 경우의 생존률을 비교할 수 있습니다.
pd.pivot_table(train, index="Single", values="Survived")

Survived
Single          
False   0.505650
True    0.303538

**12. SibSp 컬럼과  Parch 컬럼을 활용하여 가족 수(FamilySize)라는 새로운 컬럼을 만들어주세요.**

형제, 자매, 배우자(SibSp) 컬럼과 부모 자식(Parch) 컬럼은 얼핏 달라 보이지만 실은 가족 관계를 나타내는 것이라고 볼 수 있습니다. 그러므로 두 컬럼을 하나로 합쳐서 **가족 수(FamilySize)**라는 새로운 컬럼을 만들면 승객의 가족관계를 더 편리하게 분석할 수 있을 것입니다.

형제, 자매, 배우자(SibSp) 컬럼과 부모 자식(Parch) 컬럼을 더해서 가족 수(FamilySize) 컬럼을 만들어주세요. 단 가족 수를 계산할때는 언제나 나 자신을 포함해서 계산하는데, 나 자신은 SibSp 컬럼에도 Parch 컬럼에도 들어가있지 않습니다. 그러므로 가족 수(FamilySize) 컬럼은 언제나 SibSp 컬럼과 Parch 컬럼을 더한 값에서 하나가 더 많아야 합니다.

그러므로 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>SibSp</th>

        <th>Parch</th>

        <th>FamilySize</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>1</td>

        <td>0</td>

        <td>2</td>
      </tr>

      <tr>
        <th>2</th>

        <td>1</td>

        <td>0</td>

        <td>2</td>
      </tr>

      <tr>
        <th>3</th>

        <td>0</td>

        <td>0</td>

        <td>1</td>
      </tr>

      <tr>
        <th>4</th>

        <td>1</td>

        <td>0</td>

        <td>2</td>
      </tr>

      <tr>
        <th>5</th>

        <td>0</td>

        <td>0</td>

        <td>1</td>
      </tr>

      <tr>
        <th>6</th>

        <td>0</td>

        <td>0</td>

        <td>1</td>
      </tr>

      <tr>
        <th>7</th>

        <td>0</td>

        <td>0</td>

        <td>1</td>
      </tr>

      <tr>
        <th>8</th>

        <td>3</td>

        <td>1</td>

        <td>5</td>
      </tr>

      <tr>
        <th>9</th>

        <td>0</td>

        <td>2</td>

        <td>3</td>
      </tr>

      <tr>
        <th>10</th>

        <td>1</td>

        <td>0</td>

        <td>2</td>
      </tr>
    </tbody>
  </table>

또한 가족 수(FamilySize) 컬럼을 구한 뒤, 가족 수 별 생존률의 차이도 알고 싶습니다. 가족 수(ex: 1명 ~ 11명) 마다의 생존률을 구해서 출력해주세요. 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>FamilySize</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>0.303538</td>
      </tr>

      <tr>
        <th>2</th>

        <td>0.552795</td>
      </tr>

      <tr>
        <th>3</th>

        <td>0.578431</td>
      </tr>

      <tr>
        <th>4</th>

        <td>0.724138</td>
      </tr>

      <tr>
        <th>5</th>

        <td>0.200000</td>
      </tr>

      <tr>
        <th>6</th>

        <td>0.136364</td>
      </tr>

      <tr>
        <th>7</th>

        <td>0.333333</td>
      </tr>

      <tr>
        <th>8</th>

        <td>0.000000</td>
      </tr>

      <tr>
        <th>11</th>

        <td>0.000000</td>
      </tr>
    </tbody>
  </table>

In [35]:
# 형제, 자매, 배우자(SibSp) 컬럼과 부모 자식(Parch) 컬럼을 더해서 가족 수(FamilySize) 컬럼을 만듭니다.
# 또한 가족 수에 나 자신을 포함하기 위해서 언제나 +1을 해줍니다.
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# SibSp, Parch, FamilySize를 출력하여 비교합니다.
train[["SibSp", "Parch", "FamilySize"]].head(10)

(891, 25)


SibSp  Parch  FamilySize
PassengerId                          
1                1      0           2
2                1      0           2
3                0      0           1
4                1      0           2
5                0      0           1
6                0      0           1
7                0      0           1
8                3      1           5
9                0      2           3
10               1      0           2

In [36]:
# pandas의 pivot_table을 활용하여 FamilySize에 따른 생존률을 출력합니다.
# 가족 수가 1명부터 11명까지 각각의 생존률을 비교할 수 있습니다.
pd.pivot_table(train, index="FamilySize", values="Survived")

Survived
FamilySize          
1           0.303538
2           0.552795
3           0.578431
4           0.724138
5           0.200000
6           0.136364
7           0.333333
8           0.000000
11          0.000000

**13. 가족 수(FamilySize) 컬럼의 구역을 나눠주세요.**

가족 수(FamilySize) 컬럼을 기준으로 pivot_table로 분석을 해본 결과, 경우의 수가 너무 많아서(가족 수가 1명일 때 ~ 11명일 때) 분석 결과가 너무 잘게 쪼개지는 것 같습니다.

그러므로 가족 수(FamilySize) 컬럼을 세 구역으로 나누고 싶습니다. 구체적으로는 다음과 같습니다.

  * **싱글(Single)** - 동승한 가족이 아무도 없고, 나 혼자 탑승한 경우입니다.
  * **핵가족(Nuclear)** - 동승한 가족이 나 자신을 포함해 2명 이상 5명 미만인 경우입니다.
  * **대가족(Big)** - 동승한 가족이 나 자신을 포함 5명 이상인 경우입니다.
  
위의 정보를 활용하여, 가족 형태(FamilyType)라는 새로운 컬럼을 만들어 주세요. 이 컬럼에는 앞서 설명한 Single, Nuclear, 그리고 Big이 들어갑니다. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>FamilySize</th>

        <th>FamilyType</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>2</td>

        <td>Nuclear</td>
      </tr>

      <tr>
        <th>2</th>

        <td>2</td>

        <td>Nuclear</td>
      </tr>

      <tr>
        <th>3</th>

        <td>1</td>

        <td>Single</td>
      </tr>

      <tr>
        <th>4</th>

        <td>2</td>

        <td>Nuclear</td>
      </tr>

      <tr>
        <th>5</th>

        <td>1</td>

        <td>Single</td>
      </tr>

      <tr>
        <th>6</th>

        <td>1</td>

        <td>Single</td>
      </tr>

      <tr>
        <th>7</th>

        <td>1</td>

        <td>Single</td>
      </tr>

      <tr>
        <th>8</th>

        <td>5</td>

        <td>Big</td>
      </tr>

      <tr>
        <th>9</th>

        <td>3</td>

        <td>Nuclear</td>
      </tr>

      <tr>
        <th>10</th>

        <td>2</td>

        <td>Nuclear</td>
      </tr>
    </tbody>
  </table>

또한 가족 수(FamilySize)와 마찬가지로 가족 형태(FamilyType) 별 생존률의 차이도 구해주세요. 최종적으로 다음의 결과가 나와야 합니다.

 <table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>FamilyType</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>Big</th>

        <td>0.161290</td>
      </tr>

      <tr>
        <th>Nuclear</th>

        <td>0.578767</td>
      </tr>

      <tr>
        <th>Single</th>

        <td>0.303538</td>
      </tr>
    </tbody>
  </table>

In [37]:
# 가족 수(FamilSize)가 1인 승객을 가져와서, FamilyType 컬럼에 Single 이라는 값을 넣어줍니다.
train.loc[train["FamilySize"] == 1, "FamilyType"] = "Single"

# 가족 수(FamilSize)가 2 이상 5 미만인 승객을 가져와서, FamilyType 컬럼에 Nuclear(핵가족) 이라는 값을 넣어줍니다.
train.loc[(train["FamilySize"] > 1) & (train["FamilySize"] < 5), "FamilyType"] = "Nuclear"

# 가족 수(FamilSize)가 5 이상인 승객을 가져와서, FamilyType 컬럼에 Big(대가족) 이라는 값을 넣어줍니다.
train.loc[train["FamilySize"] >= 5, "FamilyType"] = "Big"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# train 데이터의 상위 10개를 띄우되, FamilySize와 FamilyType 컬럼만 출력합니다.
train[["FamilySize", "FamilyType"]].head(10)

(891, 26)


FamilySize FamilyType
PassengerId                       
1                     2    Nuclear
2                     2    Nuclear
3                     1     Single
4                     2    Nuclear
5                     1     Single
6                     1     Single
7                     1     Single
8                     5        Big
9                     3    Nuclear
10                    2    Nuclear

In [38]:
# pivot_table을 통해 가족 형태(FamilyType)의 변화에 따른 생존률을 출력합니다.
pd.pivot_table(data=train, index="FamilyType", values="Survived")

Survived
FamilyType          
Big         0.161290
Nuclear     0.578767
Single      0.303538

** 14. Single, Nuclear, Big 을 각각 One Hot Encoding 해주세요.**

앞서 작성한 코드를 분석한 결과, 가족 형태(FamilyType) 컬럼의 값이 생존자와 사망자를 판가름하는데 굉장히 중요한 영향을 차지하는 것 같습니다. 그러므로 이 컬럼값을 이번에는 이 두 개의 컬럼을 의사결정나무(Decision Tree)와 같은 머신러닝(Machine Learning) 알고리즘에 넣을 수 있도록 [One Hot Encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)을 해줄려고 합니다.

먼저 가족 수(FamilySize)를 바탕으로, 다음의 세 컬럼을 추가로 만들고 싶습니다.

  1. **FamilyType(Single)** - FamilySize가 1이면 True, 아니면 False가 들어갑니다.
  1. **FamilyType(Nuclear)** - FamilySize가 2 이상 5 미만이면 True, 아니면 False가 들어갑니다.
  1. **FamilyType(Big)** - FamilySize가 5 이상이면 True, 아니면 False가 나와야 합니다.

최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>FamilySize</th>

        <th>FamilyType(Single)</th>

        <th>FamilyType(Nuclear)</th>

        <th>FamilyType(Big)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>2</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>2</th>

        <td>2</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>3</th>

        <td>1</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>4</th>

        <td>2</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>5</th>

        <td>1</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>6</th>

        <td>1</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>7</th>

        <td>1</td>

        <td>True</td>

        <td>False</td>

        <td>False</td>
      </tr>

      <tr>
        <th>8</th>

        <td>5</td>

        <td>False</td>

        <td>False</td>

        <td>True</td>
      </tr>

      <tr>
        <th>9</th>

        <td>3</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>

      <tr>
        <th>10</th>

        <td>2</td>

        <td>False</td>

        <td>True</td>

        <td>False</td>
      </tr>
    </tbody>
  </table>

In [39]:
# 가족 수(FamilySize) 컬럼을 활용해 세 가지 구역을 나타내는 세 개의 새로운 컬럼을 만듭니다.
# 먼저 가족 수가 1명일 경우 Single 컬럼의 값에 True를, 1명이 아닐 경우 False를 대입합니다.
train["FamilyType(Single)"] = train["FamilySize"] == 1

# 이후 가족 수가 2에서 4명 사이일 경우 Nuclear 컬럼의 값에 True를, 그렇지 않을 경우 False를 대입합니다.
train["FamilyType(Nuclear)"] = (train["FamilySize"] > 1) & (train["FamilySize"] < 5)

# 마지막으로 가족 수가 5명 이상일 경우 Big 컬럼의 값에 True를, 그렇지 않을 경우 False를 대입합니다.
train["FamilyType(Big)"] = train["FamilySize"] >= 5

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# train 데이터의 상위 10개를 띄우되,
# FamilySize, FamilyType(Single), FamilyType(Nuclear), FamilyType(Big) 컬럼만 출력합니다.
train[["FamilySize", "FamilyType(Single)", "FamilyType(Nuclear)", "FamilyType(Big)"]].head(10)

(891, 29)


FamilySize  FamilyType(Single)  FamilyType(Nuclear)  \
PassengerId                                                        
1                     2               False                 True   
2                     2               False                 True   
3                     1                True                False   
4                     2               False                 True   
5                     1                True                False   
6                     1                True                False   
7                     1                True                False   
8                     5               False                False   
9                     3               False                 True   
10                    2               False                 True   

             FamilyType(Big)  
PassengerId                   
1                      False  
2                      False  
3                      False  
4                      False  
5                      False  
6                      False  
7                      False  
8                       True  
9                      False  
10                     False

또는 이런 방식으로도 One Hot Encoding을 할 수 있습니다.

In [40]:
# 가족 형태(FamilySize)가 Single이면 True, 아니면 False인 리스트를 가져옵니다.
# 이후 이 리스트를 FamilyType(Single)이라는 새로운 컬럼에 대입합니다.
train["FamilyType(Single)"] = train["FamilyType"] == "Single"

# 비슷한 방식으로 FamilyType(Nuclear)라는 새로운 컬럼을 만듭니다.
train["FamilyType(Nuclear)"] = train["FamilyType"] == "Nuclear"

# 비슷한 방식으로 FamilyType(Big)이라는 새로운 컬럼을 만듭니다.
train["FamilyType(Big)"] = train["FamilyType"] == "Big"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# FamilySize, FamilyType(Single), FamilyType(Nuclear), FamilyType(Big) 컬럼만 출력합니다.
train[["FamilySize", "FamilyType(Single)", "FamilyType(Nuclear)", "FamilyType(Big)"]].head(10)

(891, 29)


FamilySize  FamilyType(Single)  FamilyType(Nuclear)  \
PassengerId                                                        
1                     2               False                 True   
2                     2               False                 True   
3                     1                True                False   
4                     2               False                 True   
5                     1                True                False   
6                     1                True                False   
7                     1                True                False   
8                     5               False                False   
9                     3               False                 True   
10                    2               False                 True   

             FamilyType(Big)  
PassengerId                   
1                      False  
2                      False  
3                      False  
4                      False  
5                      False  
6                      False  
7                      False  
8                       True  
9                      False  
10                     False

### 이름(Name) 컬럼 분석

** 15. 이름(Name) 컬럼에서 호칭(Title) 컬럼을 뽑아주세요. **

이름(Name) 컬럼은 얼핏 분석하기 까다로워 보이지만, 실은 간단한 패턴으로 이루어져 있습니다. 일단 ```,```과 ```.```을 기준으로

  1. ```,```의 앞에 있는 부분이 성(SurName)이며,
  2. ```,```과 ```.``` 사이에 있는 부분은 승객의 호칭(Title)입니다.
  3. 마지막으로 ```.``` 뒤에 있는 부분은 이름(FirstName) 입니다.
  
가령 ```Braund, Mr. Owen Harris``` 이라는 이름의 승객은, 1) Braund라는 성을 가진, 2) Mr라는 호칭을 가진, 3) Owen Harris 라는 이름을 가진 승객으로 해석할 수 있습니다.

사용자의 이름(Name) 컬럼에서 **호칭(Title)** 컬럼을 추출해주세요. 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Name</th>

        <th>Title</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>Braund, Mr. Owen Harris</td>

        <td>Mr</td>
      </tr>

      <tr>
        <th>2</th>

        <td>Cumings, Mrs. John Bradley (Florence Briggs Th...</td>

        <td>Mrs</td>
      </tr>

      <tr>
        <th>3</th>

        <td>Heikkinen, Miss. Laina</td>

        <td>Miss</td>
      </tr>

      <tr>
        <th>4</th>

        <td>Futrelle, Mrs. Jacques Heath (Lily May Peel)</td>

        <td>Mrs</td>
      </tr>

      <tr>
        <th>5</th>

        <td>Allen, Mr. William Henry</td>

        <td>Mr</td>
      </tr>

      <tr>
        <th>6</th>

        <td>Moran, Mr. James</td>

        <td>Mr</td>
      </tr>

      <tr>
        <th>7</th>

        <td>McCarthy, Mr. Timothy J</td>

        <td>Mr</td>
      </tr>

      <tr>
        <th>8</th>

        <td>Palsson, Master. Gosta Leonard</td>

        <td>Master</td>
      </tr>

      <tr>
        <th>9</th>

        <td>Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)</td>

        <td>Mrs</td>
      </tr>

      <tr>
        <th>10</th>

        <td>Nasser, Mrs. Nicholas (Adele Achem)</td>

        <td>Mrs</td>
      </tr>
    </tbody>
  </table>

또한 호칭(Title) 별 생존률의 차이도 구해주세요. 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>Title</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>Capt</th>

        <td>0.000000</td>
      </tr>

      <tr>
        <th>Col</th>

        <td>0.500000</td>
      </tr>

      <tr>
        <th>Don</th>

        <td>0.000000</td>
      </tr>

      <tr>
        <th>Dr</th>

        <td>0.428571</td>
      </tr>

      <tr>
        <th>Jonkheer</th>

        <td>0.000000</td>
      </tr>

      <tr>
        <th>Lady</th>

        <td>1.000000</td>
      </tr>

      <tr>
        <th>Major</th>

        <td>0.500000</td>
      </tr>

      <tr>
        <th>Master</th>

        <td>0.575000</td>
      </tr>

      <tr>
        <th>Miss</th>

        <td>0.697802</td>
      </tr>

      <tr>
        <th>Mlle</th>

        <td>1.000000</td>
      </tr>

      <tr>
        <th>Mme</th>

        <td>1.000000</td>
      </tr>

      <tr>
        <th>Mr</th>

        <td>0.156673</td>
      </tr>

      <tr>
        <th>Mrs</th>

        <td>0.792000</td>
      </tr>

      <tr>
        <th>Ms</th>

        <td>1.000000</td>
      </tr>

      <tr>
        <th>Rev</th>

        <td>0.000000</td>
      </tr>

      <tr>
        <th>Sir</th>

        <td>1.000000</td>
      </tr>

      <tr>
        <th>the Countess</th>

        <td>1.000000</td>
      </tr>
    </tbody>
  </table>

In [41]:
# get_title이라는 이름의 함수를 정의합니다. 이 함수는 name이라는 변수를 인자로 받습니다.
# 이 함수는 이름을 받았을 때 이름에서 타이틀을 반환해줍니다.
# 가령 name에 "Braund, Mr. Owen Harris"가 들어오면 최종 결과는 Mr를 반환해줍니다.
def get_title(name):
    # 먼저 name을 , 을 기준으로 쪼갭니다. 쪼갠 결과는 0) Braund와 1) Mr. Owen Harris가 됩니다.
    # 여기서 1)번을 가져온 뒤 다시 . 을 기준으로 쪼갭니다. 쪼갠 결과는 0) Mr와 1) Owen Harris가 됩니다.
    # 여기서 0)번을 반환합니다. 최종적으로는 Mr를 반환하게 됩니다.
    return name.split(", ")[1].split('. ')[0]

# 모든 Name 컬럼 데이터에 get_title 함수를 적용한 뒤 그 결과를 Title이라는 이름의 새로운 컬럼에 대입합니다.
train["Title"] = train["Name"].apply(get_title)

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 이름(Name) 컬럼과 호칭(Title) 컬럼만을 출력합니다.
train[["Name", "Title"]].head(10)

(891, 30)


Name   Title
PassengerId                                                           
1                                      Braund, Mr. Owen Harris      Mr
2            Cumings, Mrs. John Bradley (Florence Briggs Th...     Mrs
3                                       Heikkinen, Miss. Laina    Miss
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)     Mrs
5                                     Allen, Mr. William Henry      Mr
6                                             Moran, Mr. James      Mr
7                                      McCarthy, Mr. Timothy J      Mr
8                               Palsson, Master. Gosta Leonard  Master
9            Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)     Mrs
10                         Nasser, Mrs. Nicholas (Adele Achem)     Mrs

In [42]:
# pivot_table을 통해 호칭(Title)별 생존률을 출력합니다.
# Mr, Mrs, Master, Miss 등의 호칭별 생존률을 확인할 수 있습니다.
pd.pivot_table(train, index="Title", values="Survived")

Survived
Title                 
Capt          0.000000
Col           0.500000
Don           0.000000
Dr            0.428571
Jonkheer      0.000000
Lady          1.000000
Major         0.500000
Master        0.575000
Miss          0.697802
Mlle          1.000000
Mme           1.000000
Mr            0.156673
Mrs           0.792000
Ms            1.000000
Rev           0.000000
Sir           1.000000
the Countess  1.000000

** 16. 호칭(Title) 컬럼을 정리해주세요. **

호칭(Title) 컬럼은 생존자와 사망자를 구문하는데 중요한 역할을 담당합니다만, 너무 종류가 많은 단점이 있습니다. 

가령 호칭(Title) 컬럼은 Mr, Mrs, Miss, Master가 전체 승객의 호칭의 대부분을 차지하며, Capt, Col, Don, Sir 등과 같은 컬럼들은 모수가 작아서 오히려 신뢰할 수 없습니다.

가령 Sir라는 호칭을 가진 승객의 생존률이 100%라고 하더라도, 전체 승객애서 오직 1명만이 Sir라는 호칭을 사용하고 있기 때문에, "Sir라는 호칭을 사용하는 사람은 언제나 생존률이 높다."고 이야기하기에는 어렵습니다.



In [43]:
# crosstab을 활용하여 각 호칭(Title)별 생존자의 총 인원수와 사망자의 총 인원수를 출력합니다.
pd.crosstab(train["Title"], train["Survived"], margins=True)

Survived        0    1  All
Title                      
Capt            1    0    1
Col             1    1    2
Don             1    0    1
Dr              4    3    7
Jonkheer        1    0    1
Lady            0    1    1
Major           1    1    2
Master         17   23   40
Miss           55  127  182
Mlle            0    2    2
Mme             0    1    1
Mr            436   81  517
Mrs            26   99  125
Ms              0    1    1
Rev             6    0    6
Sir             0    1    1
the Countess    0    1    1
All           549  342  891

그러므로 호칭 컬럼의 종류를 간소화하고 싶습니다. 호칭(Title) 컬럼에서 Mr, Mrs, Miss, Master를 제외한 나머지 값은 전부 **Other**라는 값으로 통일해주세요. 최종적으로 하위 5개의 데이터에서는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Name</th>

        <th>Title</th>

        <th>Title(Clean)</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>887</th>

        <td>Montvila, Rev. Juozas</td>

        <td>Rev</td>

        <td>Others</td>
      </tr>

      <tr>
        <th>888</th>

        <td>Graham, Miss. Margaret Edith</td>

        <td>Miss</td>

        <td>Miss</td>
      </tr>

      <tr>
        <th>889</th>

        <td>Johnston, Miss. Catherine Helen "Carrie"</td>

        <td>Miss</td>

        <td>Miss</td>
      </tr>

      <tr>
        <th>890</th>

        <td>Behr, Mr. Karl Howell</td>

        <td>Mr</td>

        <td>Mr</td>
      </tr>

      <tr>
        <th>891</th>

        <td>Dooley, Mr. Patrick</td>

        <td>Mr</td>

        <td>Mr</td>
      </tr>
    </tbody>
  </table>

또한 마찬가지로 새롭게 정리한 호칭별 생존률의 차이도 구해주세요. 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>Title(Clean)</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>Master</th>

        <td>0.575000</td>
      </tr>

      <tr>
        <th>Miss</th>

        <td>0.697802</td>
      </tr>

      <tr>
        <th>Mr</th>

        <td>0.156673</td>
      </tr>

      <tr>
        <th>Mrs</th>

        <td>0.792000</td>
      </tr>

      <tr>
        <th>Others</th>

        <td>0.444444</td>
      </tr>
    </tbody>
  </table>

In [44]:
# 이번에는 get_cleaned_title 이름의 함수를 정의합니다.
# 이 함수에서는 Mr, Mrs, Miss, Master 가 아닌 호칭은 전부 Other라고 반환합니다.
def get_cleaned_title(name):
    # 먼저 name을 , 을 기준으로 쪼갭니다. 쪼갠 결과는 0) Braund와 1) Mr. Owen Harris가 됩니다.
    # 여기서 1)번을 가져온 뒤 다시 . 을 기준으로 쪼갭니다. 쪼갠 결과는 0) Mr와 1) Owen Harris가 됩니다.
    # 여기서 0)번을 반환합니다. 최종적으로는 Mr를 반환하게 됩니다.
    # 해당 값을 title이라는 이름의 변수에 할당합니다.
    title = name.split(", ")[1].split('. ')[0]
    
    # title이라는 변수에 들어간 값이 Mr, Mrs, Miss, Master중에 하나에 해당되면
    # title 자기 자신을 반환합니다.
    if title in ["Mr", "Mrs", "Miss", "Master"]:
        return title
    # 그렇지 않다면 "Others"라는 값을 반환합니다.
    else:
        return "Others"

# 모든 Name 컬럼 데이터에 get_cleaned_title 함수를 적용한 뒤
# 그 결과를 Title(Clean)이라는 이름의 새로운 컬럼에 대입합니다.
train["Title(Clean)"] = train["Name"].apply(get_cleaned_title)

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 이름(Name) 컬럼과 Title 컬럼, 그리고 Title(Clean) 컬럼만을 출력합니다.
# 다만 이번에는 head가 아니라 tail을 활용해 마지막 다섯 컬럼만 출력합니다.
train[["Name", "Title", "Title(Clean)"]].tail()

(891, 31)


Name Title Title(Clean)
PassengerId                                                             
887                             Montvila, Rev. Juozas   Rev       Others
888                      Graham, Miss. Margaret Edith  Miss         Miss
889          Johnston, Miss. Catherine Helen "Carrie"  Miss         Miss
890                             Behr, Mr. Karl Howell    Mr           Mr
891                               Dooley, Mr. Patrick    Mr           Mr

In [45]:
# pivot_table을 통해 정리한 호칭별 생존률을 출력합니다.
# Mr, Mrs, Master, Miss, Others별 생존률을 확인할 수 있습니다.
pd.pivot_table(train, index="Title(Clean)", values="Survived")

Survived
Title(Clean)          
Master        0.575000
Miss          0.697802
Mr            0.156673
Mrs           0.792000
Others        0.444444

** 17. 호칭(Title) 컬럼으로 결혼 유무를 파악해주세요. **

앞서 우리가 추출한 호칭 컬럼의 주요 값에 대한 설명은 다음과 같습니다.

  * Mr - 결혼한 남성을 의미합니다.
  * Master - 결혼하지 않은 남성을 의미합니다.
  * Mrs - 결혼한 여성을 의미합니다.
  * Miss - 결혼하지 않은 여성을 의미합니다.
  
그러므로 호칭(Title) 컬럼값에서는 추가로 결혼 여부를 뽑아낼 수 있습니다. 호칭이 Mr이거나 Mrs이면 결혼했다고 간주하고, Master이거나 Miss이면 결혼하지 않았다고 간주할 수 있습니다. (나머지 호칭은 NaN을 집어넣으면 좋겠습니다)

호칭(Title) 컬럼을 활용하여 **결혼 유무(Married)**를 추출해주세요, 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Name</th>

        <th>Title</th>

        <th>Married</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>Braund, Mr. Owen Harris</td>

        <td>Mr</td>

        <td>True</td>
      </tr>

      <tr>
        <th>2</th>

        <td>Cumings, Mrs. John Bradley (Florence Briggs Th...</td>

        <td>Mrs</td>

        <td>True</td>
      </tr>

      <tr>
        <th>3</th>

        <td>Heikkinen, Miss. Laina</td>

        <td>Miss</td>

        <td>False</td>
      </tr>

      <tr>
        <th>4</th>

        <td>Futrelle, Mrs. Jacques Heath (Lily May Peel)</td>

        <td>Mrs</td>

        <td>True</td>
      </tr>

      <tr>
        <th>5</th>

        <td>Allen, Mr. William Henry</td>

        <td>Mr</td>

        <td>True</td>
      </tr>

      <tr>
        <th>6</th>

        <td>Moran, Mr. James</td>

        <td>Mr</td>

        <td>True</td>
      </tr>

      <tr>
        <th>7</th>

        <td>McCarthy, Mr. Timothy J</td>

        <td>Mr</td>

        <td>True</td>
      </tr>

      <tr>
        <th>8</th>

        <td>Palsson, Master. Gosta Leonard</td>

        <td>Master</td>

        <td>False</td>
      </tr>

      <tr>
        <th>9</th>

        <td>Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)</td>

        <td>Mrs</td>

        <td>True</td>
      </tr>

      <tr>
        <th>10</th>

        <td>Nasser, Mrs. Nicholas (Adele Achem)</td>

        <td>Mrs</td>

        <td>True</td>
      </tr>
    </tbody>
  </table>

또한 결혼 유무(Married)별 생존률의 차이도 구해주세요. 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>
      </tr>

      <tr>
        <th>Married</th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>False</th>

        <td>0.675676</td>
      </tr>

      <tr>
        <th>True</th>

        <td>0.280374</td>
      </tr>
    </tbody>
  </table>

In [46]:
# 호칭(Title)이 Mr인 데이터를 가져와서, 결혼 유무(Married)라는 새로운 컬럼에 True라는 값을 대입합니다.
train.loc[train["Title"] == "Mr", "Married"] = True

# 비슷하게 호칭(Title)이 Mrs인 데이터를 가져와서 결혼 유무(Married) 컬럼에 True라는 값을 대입합니다.
train.loc[train["Title"] == "Mrs", "Married"] = True

# 호칭(Title)이 Master인 데이터를 가져와서 결혼 유무(Married) 컬럼에 False라는 값을 대입합니다.
train.loc[train["Title"] == "Master", "Married"] = False

# 호칭(Title)이 Miss인 데이터를 가져와서 결혼 유무(Married) 컬럼에 False라는 값을 대입합니다.
train.loc[train["Title"] == "Miss", "Married"] = False

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 이름(Name) 컬럼과 호칭(Title) 컬럼, 그리고 결혼 유무(Married) 컬럼을 출력합니다.
train[["Name", "Title", "Married"]].head(10)

(891, 32)


Name   Title Married
PassengerId                                                                   
1                                      Braund, Mr. Owen Harris      Mr    True
2            Cumings, Mrs. John Bradley (Florence Briggs Th...     Mrs    True
3                                       Heikkinen, Miss. Laina    Miss   False
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)     Mrs    True
5                                     Allen, Mr. William Henry      Mr    True
6                                             Moran, Mr. James      Mr    True
7                                      McCarthy, Mr. Timothy J      Mr    True
8                               Palsson, Master. Gosta Leonard  Master   False
9            Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)     Mrs    True
10                         Nasser, Mrs. Nicholas (Adele Achem)     Mrs    True

또는 이런 방식으로도 결혼 유무를 파악할 수 있습니다.

In [47]:
# 호칭(Title)이 Mr이거나 Mrs인 데이터를 가져옵니다.
# 이후 결혼 유무(Married)라는 새로운 컬럼에 True라는 값을 대입합니다.
train.loc[train["Title"].isin(["Mr", "Mrs"]), "Married"] = True

# 비슷하게 호칭(Title)이 Master이거나 Miss인 데이터를 가져옵니다.
# 마찬가지로 결혼 유무(Married)라는 컬럼에 False라는 값을 대입합니다.
train.loc[train["Title"].isin(["Master", "Miss"]), "Married"] = False

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 이름(Name) 컬럼과 호칭(Title) 컬럼, 그리고 결혼 유무(Married) 컬럼을 출력합니다.
train[["Name", "Title", "Married"]].head(10)

(891, 32)


Name   Title Married
PassengerId                                                                   
1                                      Braund, Mr. Owen Harris      Mr    True
2            Cumings, Mrs. John Bradley (Florence Briggs Th...     Mrs    True
3                                       Heikkinen, Miss. Laina    Miss   False
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)     Mrs    True
5                                     Allen, Mr. William Henry      Mr    True
6                                             Moran, Mr. James      Mr    True
7                                      McCarthy, Mr. Timothy J      Mr    True
8                               Palsson, Master. Gosta Leonard  Master   False
9            Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)     Mrs    True
10                         Nasser, Mrs. Nicholas (Adele Achem)     Mrs    True

In [48]:
# pivot_table을 통해 결혼 유무(Married)별 따른 생존률을 출력합니다.
# 결혼을 한 승객(True)와 하지 않은 승객(False)간의 생존률의 차이를 볼 수 있습니다.
pd.pivot_table(train, index="Married", values="Survived")

Survived
Married          
False    0.675676
True     0.280374

**18. 성(Surname)과 가족 사이즈(FamilySize)를 조합해서 가족 ID(FamilyID)를 구해주세요.**

앞서 말씀드린대로 이름(Name) 컬럼은 1) 성(Surname), 2) 호칭(Title), 그리고 3) 이름(FirstName) 컬럼으로 나눌 수 있습니다. 가령 ```Braund, Mr. Owen Harris``` 라는 이름의 승객은, 1) Braund라는 성을 가진, 2) Mr라는 호칭을 가진, 3) Owen Harris 라는 이름을 가진 승객으로 해석할 수 있습니다.

여기서 위에서 발견한 성(Surname) 컬럼과 가족 수(FamilySize)를 앞뒤로 붙이면, 일명 가족 아이디(FamilyID)라는 새로운 컬럼을 얻을 수 있습니다.

가령 타이타닉의 train 데이터에서 성이 ```Graham```인 승객을 찾아보겠습니다.

In [49]:
# 이름(Name) 컬럼에서 Graham이라는 성이 포함된 승객만을 색인합니다.
# 이 결과를 graham이라는 이름의 변수에 저장한 후 출력합니다.
graham = train[train["Name"].str.contains("Graham")]
graham

Survived  Pclass                                           Name  \
PassengerId                                                                    
269                 1       1  Graham, Mrs. William Thompson (Edith Junkins)   
333                 0       1                      Graham, Mr. George Edward   
888                 1       1                   Graham, Miss. Margaret Edith   

                Sex   Age  SibSp  Parch    Ticket      Fare Cabin   ...     \
PassengerId                                                         ...      
269          female  58.0      0      1  PC 17582  153.4625  C125   ...      
333            male  38.0      0      1  PC 17582  153.4625   C91   ...      
888          female  19.0      0      0    112053   30.0000   B42   ...      

            Age(fill) Single FamilySize FamilyType  FamilyType(Single)  \
PassengerId                                                              
269              58.0  False          2    Nuclear               False   
333              38.0  False          2    Nuclear               False   
888              19.0   True          1     Single                True   

             FamilyType(Nuclear)  FamilyType(Big)  Title  Title(Clean)  \
PassengerId                                                              
269                         True            False    Mrs           Mrs   
333                         True            False     Mr            Mr   
888                        False            False   Miss          Miss   

             Married  
PassengerId           
269             True  
333             True  
888            False  

[3 rows x 32 columns]

색인 결과 Graham이라는 성을 사용하는 총 3명의 승객이 나오지만, 이 3명이 전부 같은 가족은 아닙니다. 이유는 가족 수(FamilySize) 컬럼을 보면 알 수 있습니다.

In [50]:
# graham이라는 변수에 저장된 데이터에서
# 이름(Name), 형제, 자매, 배우자(SibSp), 부모, 자식(Parch), 그리고 가족 수(FamilySize)를 출력합니다.
graham[["Name", "SibSp", "Parch", "FamilySize"]]

Name  SibSp  Parch  \
PassengerId                                                                
269          Graham, Mrs. William Thompson (Edith Junkins)      0      1   
333                              Graham, Mr. George Edward      0      1   
888                           Graham, Miss. Margaret Edith      0      0   

             FamilySize  
PassengerId              
269                   2  
333                   2  
888                   1

결과를 살펴보면 같은 Graham 이라는 성을 사용하더라도 가족 수가 다른 것을 알 수 있습니다. 즉, 타이타닉의 train 데이터에는 1) Graham이라는 성을 사용하는 2명의 가족, 그리고 2) 그리고 Graham라는 성을 사용하는, 가족이 없이 혼자서 탄 승객. 이렇게 두 타입의 다른 가족이 있다는 것을 알 수 있습니다.

그러므로 가족 관계를 유추하기 위해서는, 성(Surname)과 가족 수(FamilySize)를 앞뒤고 붙인 가족 아이디(FamilyID)라는 새로운 컬럼이 있다면 좋겠습니다. 가령 위의 경우에는 Graham2라는 FamilyID와 Graham1이라는 FamilyID가 생깁니다.

성(Surname)과 가족 수(FamilySize) 컬럼을 활용하여 가족 아이디(FamilyID) 컬럼을 만들어주세요. 최종적으로는 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe" style="float: left;">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Name</th>

        <th>Surname</th>

        <th>FamilySize</th>

        <th>FamilyID</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>Braund, Mr. Owen Harris</td>

        <td>Braund</td>

        <td>2</td>

        <td>Braund2</td>
      </tr>

      <tr>
        <th>2</th>

        <td>Cumings, Mrs. John Bradley (Florence Briggs Th...</td>

        <td>Cumings</td>

        <td>2</td>

        <td>Cumings2</td>
      </tr>

      <tr>
        <th>3</th>

        <td>Heikkinen, Miss. Laina</td>

        <td>Heikkinen</td>

        <td>1</td>

        <td>Heikkinen1</td>
      </tr>

      <tr>
        <th>4</th>

        <td>Futrelle, Mrs. Jacques Heath (Lily May Peel)</td>

        <td>Futrelle</td>

        <td>2</td>

        <td>Futrelle2</td>
      </tr>

      <tr>
        <th>5</th>

        <td>Allen, Mr. William Henry</td>

        <td>Allen</td>

        <td>1</td>

        <td>Allen1</td>
      </tr>
    </tbody>
  </table>

In [51]:
# get_surname이라는 이름의 함수를 만듭니다. 이 함수에는 name이라는 이름의 변수가 인자로 들어갑니다.
# 이 함수는 이름을 받았을 때 이름에서 타이틀을 반환해줍니다.
# 가령 name에 "Braund, Mr. Owen Harris"가 들어오면 최종 결과는 Braund를 반환해줍니다.
def get_surname(name):
    # 먼저 name을 , 을 기준으로 쪼갭니다. 쪼갠 결과는 0) Braund와 1) Mr. Owen Harris가 됩니다.
    # 여기서 0)번을 반환합니다. 최종적으로는 Braund를 반환하게 됩니다.
    # 해당 값을 surname이라는 이름의 변수에 할당합니다.
    surname = name.split(", ")[0]
    
    # surname을 반환합니다.
    return surname

# 모든 Name 컬럼 데이터에 get_surname 함수를 적용한 뒤
# 그 결과를 Surname이라는 이름의 새로운 컬럼에 대입합니다.
train["Surname"] = train["Name"].apply(get_surname)

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 이름(Name) 컬럼과 성(Surname) 컬럼만을 출력합니다.
train[["Name", "Surname"]].head()

(891, 33)


Name    Surname
PassengerId                                                              
1                                      Braund, Mr. Owen Harris     Braund
2            Cumings, Mrs. John Bradley (Florence Briggs Th...    Cumings
3                                       Heikkinen, Miss. Laina  Heikkinen
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)   Futrelle
5                                     Allen, Mr. William Henry      Allen

In [52]:
# 성(Surname)과 가족 수(FamilySize)를 앞뒤로 붙입니다.
# 다만 문자열(str)과 정수형(int)는 덧셈이 되지 않기 때문에,
# 정수형에 해당하는 가족 수(FamilySize) 컬럼을 .astype 함수를 이용하여 문자열로 변환합니다.
# 그리고 덧셈의 결과를 가족 아이디(FamilyID)라는 이름의 컬럼에 할당합니다.
train['FamilyID'] = train['Surname'] + train["FamilySize"].astype('str')

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 이름(Name) 컬럼과 성(Surname) 컬럼, 가족 수(FamilySize) 컬럼, 그리고 가족 아이디(FamilyID) 컬럼을 출력합니다.
train[["Name", "Surname", "FamilySize", "FamilyID"]].head()

(891, 34)


Name    Surname  \
PassengerId                                                                 
1                                      Braund, Mr. Owen Harris     Braund   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...    Cumings   
3                                       Heikkinen, Miss. Laina  Heikkinen   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)   Futrelle   
5                                     Allen, Mr. William Henry      Allen   

             FamilySize    FamilyID  
PassengerId                          
1                     2     Braund2  
2                     2    Cumings2  
3                     1  Heikkinen1  
4                     2   Futrelle2  
5                     1      Allen1

**19. 가족 아이디(FamilyID) 컬럼을 One Hot Encoding 해주세요. **

이번에는 가족 아이디(FamilyID)도 컬럼도 의사결정나무(Decision Tree)와 같은 머신러닝(Machine Learning) 알고리즘에 넣을 수 있도록 [One Hot Encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)을 해줄려고 합니다. 다만 이번에는 One Hot Encoding의 결과가 너무 많다는 것에 유의해야 합니다. 최종적으로 다음의 결과가 나와야 합니다.

<table border="1" class="dataframe">
    <thead>
      <tr style="text-align: right;">
        <th></th>

        <th>Survived</th>

        <th>Pclass</th>

        <th>Name</th>

        <th>Sex</th>

        <th>Age</th>

        <th>SibSp</th>

        <th>Parch</th>

        <th>Ticket</th>

        <th>Fare</th>

        <th>Cabin</th>

        <th>...</th>

        <th>Yousseff1</th>

        <th>Yrois1</th>

        <th>Zabour2</th>

        <th>Zimmerman1</th>

        <th>de Messemaeker2</th>

        <th>de Mulder1</th>

        <th>de Pelsmaeker1</th>

        <th>del Carlo2</th>

        <th>van Billiard3</th>

        <th>van Melkebeke1</th>
      </tr>

      <tr>
        <th>PassengerId</th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>

        <th></th>
      </tr>
    </thead>

    <tbody>
      <tr>
        <th>1</th>

        <td>0</td>

        <td>3</td>

        <td>Braund, Mr. Owen Harris</td>

        <td>male</td>

        <td>22.0</td>

        <td>1</td>

        <td>0</td>

        <td>A/5 21171</td>

        <td>7.2500</td>

        <td>NaN</td>

        <td>...</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>
      </tr>

      <tr>
        <th>2</th>

        <td>1</td>

        <td>1</td>

        <td>Cumings, Mrs. John Bradley (Florence Briggs Th...</td>

        <td>female</td>

        <td>38.0</td>

        <td>1</td>

        <td>0</td>

        <td>PC 17599</td>

        <td>71.2833</td>

        <td>C85</td>

        <td>...</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>
      </tr>

      <tr>
        <th>3</th>

        <td>1</td>

        <td>3</td>

        <td>Heikkinen, Miss. Laina</td>

        <td>female</td>

        <td>26.0</td>

        <td>0</td>

        <td>0</td>

        <td>STON/O2. 3101282</td>

        <td>7.9250</td>

        <td>NaN</td>

        <td>...</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>
      </tr>

      <tr>
        <th>4</th>

        <td>1</td>

        <td>1</td>

        <td>Futrelle, Mrs. Jacques Heath (Lily May Peel)</td>

        <td>female</td>

        <td>35.0</td>

        <td>1</td>

        <td>0</td>

        <td>113803</td>

        <td>53.1000</td>

        <td>C123</td>

        <td>...</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>
      </tr>

      <tr>
        <th>5</th>

        <td>0</td>

        <td>3</td>

        <td>Allen, Mr. William Henry</td>

        <td>male</td>

        <td>35.0</td>

        <td>0</td>

        <td>0</td>

        <td>373450</td>

        <td>8.0500</td>

        <td>NaN</td>

        <td>...</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>

        <td>0</td>
      </tr>
    </tbody>
  </table>

가족 아이디(FamilyID) 컬럼을 One Hot Encoding 한 뒤 그 결과를 train 데이터에 합쳐주세요.

In [53]:
# 가족 아이디(FamilyID) 컬럼에서 중복을 제거합니다.
# 그리고 그 결과를 family_ids라는 이름의 변수에 할당합니다.
family_ids = train["FamilyID"].unique()

# family_ids 변수에 할당된 데이터의 갯수를 출력합니다.
print(len(family_ids))

# family_ids 변수에 할당된 리스트의 상위 10개만을 출력합니다.
family_ids[0:10]

701


array(['Braund2', 'Cumings2', 'Heikkinen1', 'Futrelle2', 'Allen1',
       'Moran1', 'McCarthy1', 'Palsson5', 'Johnson3', 'Nasser2'], dtype=object)

In [54]:
# family_ids 리스트 안에 있는 모든 값을 반복문을 통해 꺼내옵니다.
# 꺼내온 값을 family_id라는 이름의 변수에 할당합니다.
for family_id in family_ids:
    # 가족 아이디(FamilyID) 컬럼값이 family_id 변수에 할당된 값과 일치하면 True, 그렇지 않으면 False를 반환합니다.
    # 그 결과를 family_id 변수에 할당된 값을 이름으로 하는 새로운 컬럼에 추가합니다.

    # 가령 family_id 변수에 Braund2가 할당되어 있을 경우,
    # 가족 아이디(FamilyID) 가 Braund2인 경우에는 True, 아니면 False를 반환하는 리스트를 만들고
    # 그 결과를 Braund2 이름의 새로운 컬럼에 할당합니다.
    train[family_id] = train["FamilyID"] == family_id
    
# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# train 데이터의 상위 5개를 출력합니다.
train.head()

(891, 735)


Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin   ...    Potter2  \
PassengerId                                                  ...              
1                1      0         A/5 21171   7.2500   NaN   ...      False   
2                1      0          PC 17599  71.2833   C85   ...      False   
3                0      0  STON/O2. 3101282   7.9250   NaN   ...      False   
4                1      0            113803  53.1000  C123   ...      False   
5                0      0            373450   8.0500   NaN   ...      False   

            Shelley2 Markun1 Dahlberg1  Banfield1  Sutehall1  Montvila1  \
PassengerId                                                               
1              False   False     False      False      False      False   
2              False   False     False      False      False      False   
3              False   False     False      False      False      False   
4              False   False     False      False      False      False   
5              False   False     False      False      False      False   

             Graham1  Behr1  Dooley1  
PassengerId                           
1              False  False    False  
2              False  False    False  
3              False  False    False  
4              False  False    False  
5              False  False    False  

[5 rows x 735 columns]

또는 pandas의 [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)와 [concat](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html)을 활용하면 다음과 같은 방식으로 가족 아이디(FamilyID) 컬럼을 One Hot Encoding 할 수 있습니다.

In [55]:
# pandas의 get_dummies를 활용합니다.
# 이 함수를 사용하면 가족 아이디(FamilyID)에 있는 모든 값을 자동으로 One Hot Encoding 해줍니다.
family_ids = pd.get_dummies(train["FamilyID"])

# family_ids 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(family_ids.shape)

# family_ids 데이터의 상위 5개를 출력합니다.
family_ids.head()

(891, 701)


Abbing1  Abbott3  Abelson2  Adahl1  Adams1  Ahlin2  Aks2  \
PassengerId                                                             
1                  0        0         0       0       0       0     0   
2                  0        0         0       0       0       0     0   
3                  0        0         0       0       0       0     0   
4                  0        0         0       0       0       0     0   
5                  0        0         0       0       0       0     0   

             Albimona1  Alexander1  Alhomaki1       ...        Yousseff1  \
PassengerId                                         ...                    
1                    0           0          0       ...                0   
2                    0           0          0       ...                0   
3                    0           0          0       ...                0   
4                    0           0          0       ...                0   
5                    0           0          0       ...                0   

             Yrois1  Zabour2  Zimmerman1  de Messemaeker2  de Mulder1  \
PassengerId                                                             
1                 0        0           0                0           0   
2                 0        0           0                0           0   
3                 0        0           0                0           0   
4                 0        0           0                0           0   
5                 0        0           0                0           0   

             de Pelsmaeker1  del Carlo2  van Billiard3  van Melkebeke1  
PassengerId                                                             
1                         0           0              0               0  
2                         0           0              0               0  
3                         0           0              0               0  
4                         0           0              0               0  
5                         0           0              0               0  

[5 rows x 701 columns]

In [56]:
# .concat 함수를 사용하여 train 데이터와 family_ids 데이터를 하나로 합칩니다.
# 여기서 axis=0을 주면 두 개의 데이터를 위-아래로 합치고, axis=1을 주면 두 개의 데이터를 왼쪽-오른쪽으로 합칩니다.
# 그 결과를 다시 train 변수에 할당합니다.
train = pd.concat([train, family_ids], axis=1)
    
# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# train 데이터의 상위 5개를 출력합니다.
train.head()

(891, 1436)


Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   

                                                          Name     Sex   Age  \
PassengerId                                                                    
1                                      Braund, Mr. Owen Harris    male  22.0   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
3                                       Heikkinen, Miss. Laina  female  26.0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
5                                     Allen, Mr. William Henry    male  35.0   

             SibSp  Parch            Ticket     Fare Cabin       ...        \
PassengerId                                                      ...         
1                1      0         A/5 21171   7.2500   NaN       ...         
2                1      0          PC 17599  71.2833   C85       ...         
3                0      0  STON/O2. 3101282   7.9250   NaN       ...         
4                1      0            113803  53.1000  C123       ...         
5                0      0            373450   8.0500   NaN       ...         

            Yousseff1 Yrois1 Zabour2 Zimmerman1  de Messemaeker2  de Mulder1  \
PassengerId                                                                    
1                   0      0       0          0                0           0   
2                   0      0       0          0                0           0   
3                   0      0       0          0                0           0   
4                   0      0       0          0                0           0   
5                   0      0       0          0                0           0   

             de Pelsmaeker1  del Carlo2  van Billiard3  van Melkebeke1  
PassengerId                                                             
1                         0           0              0               0  
2                         0           0              0               0  
3                         0           0              0               0  
4                         0           0              0               0  
5                         0           0              0               0  

[5 rows x 1436 columns]

## 마무리하며

지금까지 파이썬의 데이터 분석 패키지 판다스([Pandas](https://pandas.pydata.org/))를 활용한 실전 예제를 살펴보았습니다. 앞서 말씀드린대로, 위 문제를 실전에서 반나절(3~4시간) 안에 해결할 수 있다면 현업에서 데이터 사이언티스트로서 일 할 수 있는 충분한 판다스 스킬을 보유했다고 볼 수 있습니다.

판다스를 추가 학습자료를 추천합니다.

  * [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html)
  * [Pandas Cookbook](http://github.com/jvns/pandas-cookbook)
  * [Python for Data Science](http://wavedatalab.github.io/datawithpython/)
  * [Modern Pandas](http://tomaugspurger.github.io/modern-1-intro.html)